In [1]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from scipy.optimize import minimize_scalar


class RandomForestMSE:
    def __init__(self, n_estimators, max_depth=None, feature_subsample_size=None,
                 **trees_parameters):
        """
        n_estimators : int
            The number of trees in the forest.
        
        max_depth : int
            The maximum depth of the tree. If None then there is no limits.
        
        feature_subsample_size : float
            The size of feature set for each tree. If None then use recommendations.
        """

        self.n_estimators = n_estimators
        self.max_depth = max_depth
        if feature_subsample_size is None:
            self.feature_subsample_size = 1
        else:
            self.feature_subsample_size = feature_subsample_size
        self.trees_parameters = trees_parameters
        
    def fit(self, X, y):
        """
        X : numpy ndarray
            Array of size n_objects, n_features
            
        y : numpy ndarray
            Array of size n_objects
        """

        alpha = 1

        indexes_obj_all = np.arange(X.shape[0])
        indexes_feat_all = np.arange(X.shape[1])
        self.tree_models = []
        for i in range(self.n_estimators):
            indexes_obj_subset = np.random.choice(indexes_obj_all,
                                                  size=int(alpha * X.shape[0]),
                                                  replace=False)
#             indexes_feat_subset = np.random.choice(indexes_feat_all,
#                                                    size=int(self.feature_subsample_size * X.shape[1]),
#                                                    replace=False)
            dec_tree = None
            dec_tree = DecisionTreeRegressor(*self.trees_parameters,
                                             max_depth=self.max_depth,
                                             max_features=self.feature_subsample_size
                                             )
            dec_tree.fit(X[indexes_obj_subset],
                         y[indexes_obj_subset])
            self.tree_models.append(dec_tree)
            del dec_tree




        
    def predict(self, X):
        """
        X : numpy ndarray
            Array of size n_objects, n_features
            
        Returns
        -------
        y : numpy ndarray
            Array of size n_objects
        """

        preds = np.zeros(X.shape[0])
        for i in range(self.n_estimators):
            preds += self.tree_models[i].predict(X) / self.n_estimators
        return preds



class GradientBoostingMSE:
    def __init__(self, n_estimators, learning_rate=0.1, max_depth=5, feature_subsample_size=None,
                 **trees_parameters):
        """
        n_estimators : int
            The number of trees in the forest.
        
        learning_rate : float
            Use learning_rate * gamma instead of gamma

        max_depth : int
            The maximum depth of the tree. If None then there is no limits.
        
        feature_subsample_size : float
            The size of feature set for each tree. If None then use recommendations.
        """
        
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
#         if 
        self.feature_subsample_size = feature_subsample_size
        
        
        
    def fit(self, X, y):
        """
        X : numpy ndarray
            Array of size n_objects, n_features
            
        y : numpy ndarray
            Array of size n_objects
        """
        pass

    def predict(self, X):
        """
        X : numpy ndarray
            Array of size n_objects, n_features
            
        Returns
        -------
        y : numpy ndarray
            Array of size n_objects
        """
        pass


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [3]:
import pandas as pd

train_data = pd.read_csv('./data/data.csv', index_col=0)
target = pd.read_csv('./data/target.csv', index_col=0)

Удалим id, так как он не несет никакую информацию для модели

In [4]:
train_data.drop(columns='id', inplace=True)
train_data.drop(columns='date', inplace=True)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(train_data, target, train_size=0.8,
                                                    random_state=13)

In [6]:
rand_forest_my = RandomForestMSE(n_estimators=100, max_depth=10, feature_subsample_size=0.75)
rand_forest_sklearn = RandomForestRegressor(n_estimators=100, max_depth=10)

In [7]:
from sklearn.metrics import mean_squared_error

In [8]:
%%time
rand_forest_my.fit(X_train.values, y_train.values)

CPU times: user 5.14 s, sys: 3.23 ms, total: 5.15 s
Wall time: 5.15 s


In [9]:
preds_my = rand_forest_my.predict(X_test.values)

In [10]:
np.sqrt(mean_squared_error(y_test, preds_my))

351490.8075714239

In [11]:
%%time
rand_forest_sklearn.fit(X_train.values, y_train.values)

/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


CPU times: user 4.63 s, sys: 0 ns, total: 4.63 s
Wall time: 4.63 s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [12]:
preds_sklearn = rand_forest_sklearn.predict(X_test.values)

In [13]:
np.sqrt(mean_squared_error(y_test, preds_sklearn))

347739.65145936573